In [8]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import cv2
import os

In [9]:
# Helper function to load and preprocess images
def load_and_preprocess_image(path, target_size=(224, 224)):
    image = cv2.imread(path)
    image = cv2.resize(image, target_size)
    image = preprocess_input(image)
    return image

In [10]:
# Path definitions
real_dir = "/content/drive/MyDrive/RealFake.zip (Unzipped Files)/real_and_fake_face/training_real/"
fake_dir = "/content/drive/MyDrive/RealFake.zip (Unzipped Files)/real_and_fake_face/training_fake/"

real_paths = [os.path.join(real_dir, fname) for fname in os.listdir(real_dir)]
fake_paths = [os.path.join(fake_dir, fname) for fname in os.listdir(fake_dir)]

real_labels = [1] * len(real_paths)
fake_labels = [0] * len(fake_paths)

all_paths = real_paths + fake_paths
all_labels = real_labels + fake_labels

In [11]:
# Split data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(all_paths, all_labels, test_size=0.2, random_state=42)

In [12]:
# Create sequences manually
def create_image_sequences(image_paths, labels, seq_length=10, image_size=(224, 224)):
    sequences = []
    sequence_labels = []
    for i in range(0, len(image_paths) - seq_length + 1, seq_length):
        sequence = [load_and_preprocess_image(image_paths[j], target_size=image_size) for j in range(i, i + seq_length)]
        sequences.append(np.array(sequence))
        sequence_labels.append(labels[i])
    return np.array(sequences), np.array(sequence_labels)

In [13]:
# Prepare sequences
train_sequences, train_sequence_labels = create_image_sequences(train_paths, train_labels)
val_sequences, val_sequence_labels = create_image_sequences(val_paths, val_labels)

In [14]:
# Load models
model_inception = load_model('/content/drive/MyDrive/inceptionV3Deepfake_model.h5')
model_MobileNet = load_model('/content/drive/MyDrive/MobileNetV2Deepfake_model.h5')
model_Hybrid = load_model('/content/drive/MyDrive/hybrid_inception_lstm_model.h5')

In [17]:
# Evaluate models
def evaluate_model(model, val_sequences, val_sequence_labels):
    results = model.evaluate(val_sequences, val_sequence_labels, verbose=0) # Evaluate the model using the original val_sequences, not the reshaped one
    return results

# Evaluate models using the original sequences
results_inception = evaluate_model(model_inception, val_sequences, val_sequence_labels)
results_mobileNet = evaluate_model(model_MobileNet, val_sequences, val_sequence_labels)
results_hybrid = evaluate_model(model_Hybrid, val_sequences, val_sequence_labels)

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1917, in test_step
        y_pred = self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Inpu

In [ ]:
# Collect results for comparison
model_names = ["Inception LSTM", "Simple CNN", "Custom Model"]
accuracies = [results_inception[1], results_mobileNet[1], results_hybrid[1]]
losses = [results_inception[0], results_mobileNet[0], results_hybrid[0]]

In [ ]:
# Visualization
def plot_final_accuracy(model_names, accuracies):
    plt.figure(figsize=(12, 8))
    plt.bar(model_names, accuracies, color=['blue', 'green', 'red'])
    plt.xlabel('Model')
    plt.ylabel('Accuracy')
    plt.title('Final Validation Accuracy of Models')
    plt.ylim(0, 1)
    plt.show()

def plot_final_loss(model_names, losses):
    plt.figure(figsize=(12, 8))
    plt.bar(model_names, losses, color=['blue', 'green', 'red'])
    plt.xlabel('Model')
    plt.ylabel('Loss')
    plt.title('Final Validation Loss of Models')
    plt.show()

In [ ]:
# Plot results
plot_final_accuracy(model_names, accuracies)
plot_final_loss(model_names, losses)


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D, LSTM, TimeDistributed, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
import os

# Helper function to load and preprocess images
def load_and_preprocess_image(path, target_size=(224, 224)):
    image = cv2.imread(path)
    image = cv2.resize(image, target_size)
    image = preprocess_input(image)
    return image

# Path definitions
real_dir = "/content/drive/MyDrive/RealFake.zip (Unzipped Files)/real_and_fake_face/training_real/"
fake_dir = "/content/drive/MyDrive/RealFake.zip (Unzipped Files)/real_and_fake_face/training_fake/"

real_paths = [os.path.join(real_dir, fname) for fname in os.listdir(real_dir)]
fake_paths = [os.path.join(fake_dir, fname) for fname in os.listdir(fake_dir)]

real_labels = [1] * len(real_paths)
fake_labels = [0] * len(fake_paths)

all_paths = real_paths + fake_paths
all_labels = real_labels + fake_labels

# Split data into training and validation sets
from sklearn.model_selection import train_test_split
train_paths, val_paths, train_labels, val_labels = train_test_split(all_paths, all_labels, test_size=0.2, random_state=42)

# Create sequences manually
def create_image_sequences(image_paths, labels, seq_length=10, image_size=(224, 224)):
    sequences = []
    sequence_labels = []
    for i in range(0, len(image_paths) - seq_length + 1, seq_length):
        sequence = [load_and_preprocess_image(image_paths[j], target_size=image_size) for j in range(i, i + seq_length)]
        sequences.append(np.array(sequence))
        sequence_labels.append(labels[i])
    return np.array(sequences), np.array(sequence_labels)

train_sequences, train_sequence_labels = create_image_sequences(train_paths, train_labels)
val_sequences, val_sequence_labels = create_image_sequences(val_paths, val_labels)

# Function to create and compile a model
def create_model(model_type, input_shape):
    model_input = Input(shape=input_shape)
    if model_type == "inception_lstm":
        base_model = InceptionV3(include_top=False, weights="imagenet", input_shape=image_size + (3,))
        for layer in base_model.layers:
            layer.trainable = False
        cnn = TimeDistributed(base_model)(model_input)
        cnn = TimeDistributed(GlobalAveragePooling2D())(cnn)
        lstm = LSTM(256, return_sequences=True)(cnn)
        lstm = LSTM(256)(lstm)
        fc = Dense(512, activation="relu")(lstm)
        fc = Dropout(0.5)(fc)
        fc = Dense(256, activation="relu")(fc)
        fc = Dropout(0.5)(fc)
        output = Dense(1, activation="sigmoid")(fc)
    elif model_type == "simple_cnn":
        base_model = InceptionV3(include_top=False, weights="imagenet", input_shape=image_size + (3,))
        for layer in base_model.layers:
            layer.trainable = False
        cnn = TimeDistributed(base_model)(model_input)
        cnn = TimeDistributed(GlobalAveragePooling2D())(cnn)
        flatten = TimeDistributed(Dense(512, activation="relu"))(cnn)
        output = Dense(1, activation="sigmoid")(flatten)
    else:  # Custom model example
        base_model = InceptionV3(include_top=False, weights="imagenet", input_shape=image_size + (3,))
        for layer in base_model.layers:
            layer.trainable = False
        cnn = TimeDistributed(base_model)(model_input)
        cnn = TimeDistributed(GlobalAveragePooling2D())(cnn)
        lstm = LSTM(128, return_sequences=True)(cnn)
        lstm = LSTM(128)(lstm)
        fc = Dense(256, activation="relu")(lstm)
        fc = Dropout(0.5)(fc)
        fc = Dense(128, activation="relu")(fc)
        fc = Dropout(0.5)(fc)
        output = Dense(1, activation="sigmoid")(fc)

    model = Model(inputs=model_input, outputs=output)
    opt = SGD(learning_rate=0.01)
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model

# Training and evaluating the models
def train_and_evaluate(model_type, train_sequences, train_sequence_labels, val_sequences, val_sequence_labels):
    seq_length = 10
    image_size = (224, 224)
    input_shape = (seq_length, *image_size, 3)
    model = create_model(model_type, input_shape)

    def scheduler(epoch):
        if epoch <= 1:
            return 1.0
        elif epoch > 2 and epoch <= 10:
            return 0.1
        elif epoch > 10 and epoch <= 15:
            return 0.01
        else:
            return 0.001

    lr_callbacks = LearningRateScheduler(scheduler)

    hist = model.fit(train_sequences, train_sequence_labels,
                     epochs=20,
                     batch_size=8,
                     callbacks=[lr_callbacks],
                     validation_data=(val_sequences, val_sequence_labels))

    return hist.history

# Compare 3 models
history_inception_lstm = train_and_evaluate("inception_lstm", train_sequences, train_sequence_labels, val_sequences, val_sequence_labels)
history_simple_cnn = train_and_evaluate("simple_cnn", train_sequences, train_sequence_labels, val_sequences, val_sequence_labels)
history_custom_model = train_and_evaluate("custom_model", train_sequences, train_sequence_labels, val_sequences, val_sequence_labels)

# Visualization
def plot_history(histories, title):
    plt.figure(figsize=(12, 8))
    for name, history in histories.items():
        plt.plot(history['accuracy'], label=f'{name} train accuracy')
        plt.plot(history['val_accuracy'], label=f'{name} val accuracy')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

def plot_loss(histories, title):
    plt.figure(figsize=(12, 8))
    for name, history in histories.items():
        plt.plot(history['loss'], label=f'{name} train loss')
        plt.plot(history['val_loss'], label=f'{name} val loss')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_final_accuracy(histories):
    final_train_accuracies = [history['accuracy'][-1] for history in histories.values()]
    final_val_accuracies = [history['val_accuracy'][-1] for history in histories.values()]
    model_names = list(histories.keys())

    plt.figure(figsize=(12, 8))
    bar_width = 0.35
    index = np.arange(len(model_names))

    plt.bar(index, final_train_accuracies, bar_width, label='Train Accuracy')
    plt.bar(index + bar_width, final_val_accuracies, bar_width, label='Val Accuracy')

    plt.xlabel('Model')
    plt.ylabel('Accuracy')
    plt.title('Final Training and Validation Accuracy')
    plt.xticks(index + bar_width / 2, model_names)
    plt.legend()
    plt.show()

histories = {
    "Inception LSTM": history_inception_lstm,
    "Simple CNN": history_simple_cnn,
    "Custom Model": history_custom_model
}

plot_history(histories, "Model Accuracy Comparison")
plot_loss(histories, "Model Loss Comparison")
plot_final_accuracy(histories)
